##### AIT 726 - Final Project Naive Bayes Code
##### By Japneet Kohli

####  The code is executed in the following order.

1.   Load data.
2.   Select only translation writer and line translations data from the list of attributes.
3.   Assign labels of translation writer attribute binary numeric values (0,1).
4.   Preprocess text using regex.
5.   Split data into train, validation, and test.
6.   Create pipeline with functions including Logistic Regression, Count Vectorizer, and TF-IDF Transformer, and assign range of hyerparameters to be tuned.
7.   Perform 10 fold cross validation using GridSearchCV to find best values for hyperparameters. 
8.   Fit the model with the tuned parameters on training data.
9.   Predict values for validation and test data and compute accuracy using classfication report and confusion matrix.

In [31]:
# import required packages
import nltk
import pandas as pd
import numpy as np
import os
import odbc
import re
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import itertools
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
stemmer = PorterStemmer()
import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline
#from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report
print('Done')

Done


In [3]:
# load data
directory = r'csv_output_files'
dfpath = directory + "\\" + 'datadf.csv'
data=pd.read_csv(dfpath)
print('Done')

Done


In [4]:
# display data
data.head()

,Unnamed: 0,writer,lines_translations_english_writer,lines_translations_english,lines_type
0,1,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...,Manglacharan
1,2,Guru Nanak Dev Ji,Bhai Manmohan Singh,"There is but one God. True is His Name, creati...",Manglacharan
2,5,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,Chant And Meditate:,Sirlekh
3,6,Guru Nanak Dev Ji,Bhai Manmohan Singh,Embrace His meditation.,Sirlekh
4,9,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...,Pankti


In [7]:
# remove unnecessary columns from df
columns_to_keep = ['writer','lines_translations_english_writer','lines_translations_english','lines_type']
newdata=data[columns_to_keep]

In [8]:
# display data
newdata.head()

,writer,lines_translations_english_writer,lines_translations_english,lines_type
0,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...,Manglacharan
1,Guru Nanak Dev Ji,Bhai Manmohan Singh,"There is but one God. True is His Name, creati...",Manglacharan
2,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,Chant And Meditate:,Sirlekh
3,Guru Nanak Dev Ji,Bhai Manmohan Singh,Embrace His meditation.,Sirlekh
4,Guru Nanak Dev Ji,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...,Pankti


In [9]:
# describe data
newdata.describe()

,writer,lines_translations_english_writer,lines_translations_english,lines_type
count,121020,121020,121020,121020
unique,35,2,112029,4
top,Guru Arjan Dev Ji,Dr. Sant Singh Khalsa,One Universal Creator God. By The Grace Of The...,Pankti
freq,49746,60510,525,100400


In [10]:
# display data info
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121020 entries, 0 to 121019
Data columns (total 4 columns):
writer                               121020 non-null object
lines_translations_english_writer    121020 non-null object
lines_translations_english           121020 non-null object
lines_type                           121020 non-null object
dtypes: object(4)
memory usage: 3.7+ MB


In [12]:
# display data shape
newdata.shape

(121020, 4)

In [13]:
# display data value counts for translator writer
newdata.lines_translations_english_writer.value_counts()

Dr. Sant Singh Khalsa    60510
Bhai Manmohan Singh      60510
Name: lines_translations_english_writer, dtype: int64

In [14]:
# keep only required columns in df
translator_columns=['lines_translations_english_writer','lines_translations_english']
translator_data=newdata[translator_columns]

In [15]:
# display data
translator_data.head()

,lines_translations_english_writer,lines_translations_english
0,Dr. Sant Singh Khalsa,One Universal Creator God. The Name Is Truth. ...
1,Bhai Manmohan Singh,"There is but one God. True is His Name, creati..."
2,Dr. Sant Singh Khalsa,Chant And Meditate:
3,Bhai Manmohan Singh,Embrace His meditation.
4,Dr. Sant Singh Khalsa,True In The Primal Beginning. True Throughout ...


In [16]:
# display data shape
translator_data.shape

(121020, 2)

In [17]:
# describe data
translator_data.describe()

,lines_translations_english_writer,lines_translations_english
count,121020,121020
unique,2,112029
top,Dr. Sant Singh Khalsa,One Universal Creator God. By The Grace Of The...
freq,60510,525


In [18]:
# display data info
translator_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121020 entries, 0 to 121019
Data columns (total 2 columns):
lines_translations_english_writer    121020 non-null object
lines_translations_english           121020 non-null object
dtypes: object(2)
memory usage: 1.8+ MB


In [19]:
# display data for random translation text
translator_data.lines_translations_english[14]

'Hundreds of thousands of clever tricks, but not even one of them will go along with you in the end.'

In [20]:
# display data for all translation text
translator_data["lines_translations_english"]

0         One Universal Creator God. The Name Is Truth. ...
1         There is but one God. True is His Name, creati...
2                                       Chant And Meditate:
3                                   Embrace His meditation.
4         True In The Primal Beginning. True Throughout ...
                                ...                        
121015    These are the names of the sons of Maygh Raag....
121016    So all together, they sing the six Raagas and ...
121017    So all together, they sing the six Raagas and ...
121018    and all the forty-eight sons of the Raagas. ||...
121019    and all the forty-eight sons of the Raagas. ||...
Name: lines_translations_english, Length: 121020, dtype: object

In [21]:
# assign binary labels for translator writers
translator_data["lines_translations_english_writer"].replace({"Dr. Sant Singh Khalsa": "0", "Bhai Manmohan Singh": "1"}, inplace=True)

C:\Users\Japneet\Anaconda3\lib\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [22]:
# display binary labels assigned 
translator_data.lines_translations_english_writer.values

array(['0', '1', '0', ..., '1', '0', '1'], dtype=object)

In [23]:
# display data value counts for binary labels
translator_data.lines_translations_english_writer.value_counts()

0    60510
1    60510
Name: lines_translations_english_writer, dtype: int64

In [24]:
# function for data preprocessing
# remove ounctuation, html tags, non ascii characters, etc.

def preprocess_text(txt):
        txt = re.sub(r'(?:<[^>]+>)', '', txt)#Html tags
        txt = re.sub('[^\x00-\x7F]+', ' ', txt) ##remove non ascii charcters
        txt = re.sub('#\S+', '', txt)
        txt = re.sub('_', '', txt)
        txt = re.sub('-',' ',txt)
        txt = re.sub('/','',txt)
        txt = re.sub('~','',txt)
        txt = re.sub('|','',txt)
        txt = re.sub('\s+', ' ', txt) ## remove double spaces
        txt = re.sub(r'([A-Z][a-z]+)',lambda x:x.group().lower(),txt)##Lower case captilized words only but not words with all capitals
        return txt.strip()
data = [preprocess_text(t) for t in translator_data["lines_translations_english"]]
labels= translator_data.lines_translations_english_writer.values
print('Done')

Done


In [25]:
# split data into train and test; 80% train and 20% test for binomial analysis
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=42)
print('Done')

Done


In [26]:
# check size of train, test, and val datasets
print(len(x_train),len(x_test),len(x_val))

77452 24204 19364


In [35]:
# creating pipeline to obtain best hyperparameters
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('lr', LogisticRegression())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'lr__solver': ['lbfgs'],
    'lr__max_iter': [220, 1000],
    'lr__multi_class' : ('multinomial','ovr')

}

In [36]:
# obtaining best hyperparameters using GridSearchCV for binomial analysis
# fit model 
score = 'f1_micro'
print("# Tuning hyper-parameters for %s" % score)
print()
np.errstate(divide='ignore')
clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score)
clf.fit(x_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
                             clf.cv_results_['std_test_score'], 
                             clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")

# Tuning hyper-parameters for f1_micro



C:\Users\Japneet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Japneet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Japneet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Japneet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Japneet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", 

Best parameters set found on development set:

{'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

Grid scores on development set:

0.890 (+/-0.007) for {'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
0.918 (+/-0.006) for {'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
0.891 (+/-0.005) for {'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': True, 'vect__ngram_range': (2, 2)}
0.882 (+/-0.006) for {'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
0.912 (+/-0.006) for {'lr__max_iter': 220, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
0.886 (+/-0.004) for

In [37]:
# predictions using best parameters for validation and test data
y_val_pred= clf.predict(x_val)
y_test_pred = clf.predict(x_test)
print(classification_report(y_val, y_val_pred, digits=4))
print(classification_report(y_test, y_test_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9205    0.9187    0.9196      9706
           1     0.9185    0.9203    0.9194      9658

    accuracy                         0.9195     19364
   macro avg     0.9195    0.9195    0.9195     19364
weighted avg     0.9195    0.9195    0.9195     19364

              precision    recall  f1-score   support

           0     0.9173    0.9210    0.9192     12006
           1     0.9220    0.9183    0.9201     12198

    accuracy                         0.9196     24204
   macro avg     0.9196    0.9197    0.9196     24204
weighted avg     0.9197    0.9196    0.9196     24204



In [38]:
# obtain confusion matrix for test data
print(metrics.confusion_matrix(y_test, y_test_pred))

[[11058   948]
 [  997 11201]]
